### Run Scraping Function from National Pokedex Notebook

In [2]:
# To do:
    # Document widget code 

In [6]:
%run 'National Pokedex.ipynb'

In [7]:
import pickle
from IPython.display import display, clear_output
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
# Maximize display of all dataframes
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

### Scrape and Apply Transformations to Pokémon Natures Table from Serebii.net Online Database

In [9]:
natures_url = 'https://www.serebii.net/games/natures.shtml'
scrape_table(natures_url)
natures = df
# Apply transformations
natures.columns = natures.iloc[0]
natures = natures.iloc[1:]
natures = natures.rename(columns = {'Natures Name':'Nature', 'Raises':'Raised Stat', 'Lowers':'Lowered Stat', 
                                    'Loved Pokéblock':'Favorite Poffin', 
                                    'Hated Pokéblock':'Least Favorite Poffin'}).drop(['Berry','Gen 1 Remainder'], axis=1)
columns = ['Raised Stat', 'Lowered Stat', 'Favorite Poffin', 'Least Favorite Poffin']
for column in columns:
    natures[column] = natures[column].str.replace('None', 'Neutral')
natures['Nature'] = natures['Nature'].str.replace(r'[^\x00-\x7F]+', '', regex=True).str.replace('\t', '')
natures

,Nature,Raised Stat,Lowered Stat,Favorite Poffin,Least Favorite Poffin
1,Hardy,Neutral,Neutral,Neutral,Neutral
2,Lonely,Attack,Defense,Spicy (Cool),Sour (Tough)
3,Brave,Attack,Speed,Spicy (Cool),Sweet (Cute)
4,Adamant,Attack,Special Attack,Spicy (Cool),Dry (Beauty)
5,Naughty,Attack,Special Defense,Spicy (Cool),Bitter (Smart)
6,Bold,Defense,Attack,Sour (Tough),Spicy (Cool)
7,Docile,Neutral,Neutral,Neutral,Neutral
8,Relaxed,Defense,Speed,Sour (Tough),Sweet (Cute)
9,Impish,Defense,Special Attack,Sour (Tough),Dry (Beauty)
10,Lax,Defense,Special Defense,Sour (Tough),Bitter (Smart)


### Scrape and Apply Transformations to Move Tables Broken Down by Type

#### Looping through each table and storing it in a dictionary is efficient if all data is needed at once or for data analysis (efficient and flexible, however can be cost intentive)

#### FireRed/LeafGreen, Ruby/Sapphire/Emerald - Gen3

In [13]:
gen3_move_types = ['bug','dark','dragon','electric','fighting','fire','flying','ghost','grass','ground','ice','normal','poison','psychict','rock','water','steel']
gen3_moves = {}
for move_type in gen3_move_types:
    gen3_moves_url=f'https://www.serebii.net/attackdex/{move_type}.shtml'
    scrape_table(gen3_moves_url)
    # Apply transformations
    df.columns = df.iloc[6].str.strip()
    df = df.iloc[7:].rename(columns = {'Att.':'Power', 'Acc.':'Accuracy'})
    df = df[['Name', 'PP', 'Power', 'Accuracy', 'Effect']]
    df = df[~df['Name'].str.contains('All Content')]
    df['Power'] = df['Power'].replace('--', pd.NA).fillna(0).astype('int')  # Reformat moves with no damage output to allow for sorting
    df = df.sort_values('Power', ascending=True)
    df['Power'] = df['Power'].replace(0, '--')
    gen3_moves[move_type] = df
gen3_moves['psychic'] = gen3_moves.pop('psychict')

In [14]:
# Call move tables by type from dictionary
gen3_moves['dragon']

6,Name,PP,Power,Accuracy,Effect
8,Dragon Dance,20,--,--,A mystical dance that ups ATTACK and SPEED.
9,Dragon Rage,10,--,100,Launches shock waves that always inflict 40 HP damage.
12,Twister,20,40,100,Whips up a vicious twister to tear at the foe.
10,Dragonbreath,20,60,100,Strikes the foe with an incredible blast of breath.
7,Dragon Claw,15,80,100,Slashes the foe with sharp claws.
11,Outrage,15,90,100,A rampage of 2 to 3 turns that confuses the user.


#### Diamond/Pearl/Platinum, HeartGold/SoulSilver - Gen4

##### Since physical and special attacks for each type were first introduced in gen4, I am adding an additional column to specify such

In [17]:
gen4_move_types = ['bug','dark','dragon','electric','fighting','fire','flying','ghost','grass','ground','ice','normal','poison','psychict','rock','water','steel']
gen4_moves = {}
for move_type in gen4_move_types:
    gen4_moves_url=f'https://www.serebii.net/attackdex-dp/{move_type}.shtml'
    scrape_table(gen4_moves_url)
    # Apply transformations
    df.columns = df.iloc[4].str.strip()
    df = df.iloc[5:].rename(columns = {'Cat.':'Category', 'Att.':'Power', 'Acc.':'Accuracy'})
    df = df[['Name', 'Category', 'PP', 'Power', 'Accuracy', 'Effect']]
    df = df[~df['Name'].str.contains('All Content')]
    df['Category'] = df['Category'].replace('https://www.serebii.net/pokedex-dp/type/physical.png', 'Physical')\
                                .replace('https://www.serebii.net/pokedex-dp/type/special.png', 'Special')\
                                .replace('https://www.serebii.net/pokedex-dp/type/other.png', 'Other') 
    df['Power'] = df['Power'].replace('--', pd.NA).fillna(0).astype('int')  # Reformat moves with no damage output to allow for sorting
    df = df.sort_values('Power', ascending=True)
    df['Power'] = df['Power'].replace(0, '--')
    gen4_moves[move_type] = df
gen4_moves['psychic'] = gen4_moves.pop('psychict')

In [18]:
# Call gen4 move tables by type
gen4_moves['psychic']

4,Name,Category,PP,Power,Accuracy,Effect
5,Agility,Other,30,--,--,The user relaxes and lightens its body to move faster. It sharply boosts the Speed stat.
44,Trick Room,Other,5,--,--,The user creates a bizarre area in which slower Pokémon get to move first for five turns.
26,Meditate,Other,40,--,--,The user meditates to awaken the power deep within its body and raise its Attack stat.
27,Miracle Eye,Other,40,--,--,Enables the user to hit a Dark type with any type of move. It also enables the user to hit an evasive foe.
28,Mirror Coat,Special,20,--,100,"A retaliation move that counters any special attack, inflicting double the damage taken."
30,Power Swap,Other,10,--,--,The user employs its psychic power to switch changes to its Attack and Sp. Atk with the foe.
31,Power Trick,Other,10,--,--,The user employs its psychic power to switch its Attack with its Defense stat.
23,Lunar Dance,Other,10,--,--,"The user faints. In return, the Pokémon taking its place will have its status and HP fully restored."
36,Psycho Shift,Other,10,--,90,"Using its psychic power of suggestion, the user transfers its status problems to the target."
38,Reflect,Other,20,--,--,A wondrous wall of light is put up to suppress damage from physical attacks for five turns.


#### Black/White, Black/White 2 - Gen5

In [20]:
gen5_move_types = ['bug','dark','dragon','electric','fighting','fire','flying','ghost','grass','ground','ice','normal','poison','psychict','rock','water','steel']
gen5_moves = {}
for move_type in gen5_move_types:
    gen5_moves_url=f'https://www.serebii.net/attackdex-bw/{move_type}.shtml'
    scrape_table(gen5_moves_url)
    # Apply transformations
    df.columns = df.iloc[3].str.strip()
    df = df.iloc[4:].rename(columns = {'Cat.':'Category', 'Att.':'Power', 'Acc.':'Accuracy'})
    df = df[['Name', 'PP', 'Category', 'Power', 'Accuracy', 'Effect']]
    df = df[~df['Name'].str.contains('All Content')]
    df['Category'] = df['Category'].replace('https://www.serebii.net/pokedex-dp/type/physical.png', 'Physical')\
                                .replace('https://www.serebii.net/pokedex-dp/type/special.png', 'Special')\
                                .replace('https://www.serebii.net/pokedex-dp/type/other.png', 'Other')
    df['Power'] = df['Power'].replace('--', pd.NA).fillna(0).astype('int')  # Reformat moves with no damage output to allow for sorting
    df = df.sort_values('Power', ascending=True)
    df['Power'] = df['Power'].replace(0, '--')
    gen5_moves[move_type] = df
gen5_moves['psychic'] = gen5_moves.pop('psychict')

In [21]:
# Call gen5 move tables by type
gen5_moves['psychic']

3,Name,PP,Category,Power,Accuracy,Effect
4,Agility,30,Other,--,--,The user relaxes and lightens its body to move faster. It sharply boosts the Speed stat.
28,Magic Coat,15,Other,--,--,A barrier reflects back to the target moves like Leech Seed and moves that damage status.
29,Magic Room,10,Other,--,--,The user creates a bizarre area in which Pokémon's held items lose their effects for five turns.
55,Wonder Room,10,Other,--,--,The user creates a bizarre area in which Pokémon's Defense and Sp. Def stats are swapped for five turns.
31,Miracle Eye,40,Other,--,--,Enables a Dark-type target to be hit by Psychic-type attacks. It also enables an evasive target to be hit.
32,Mirror Coat,20,Special,--,100,"A retaliation move that counters any special attack, inflicting double the damage taken."
34,Power Split,10,Other,--,--,The user employs its psychic power to average its Attack and Sp. Atk stats with those of the target's.
35,Power Swap,10,Other,--,--,The user employs its psychic power to switch changes to its Attack and Sp. Atk with the target.
36,Power Trick,10,Other,--,--,The user employs its psychic power to switch its Attack with its Defense stat.
26,Lunar Dance,10,Other,--,--,"The user faints. In return, the Pokémon taking its place will have its status and HP fully restored."


### Scrape and Apply Transformations to Pokemon Moveset Tables for Each Generation

In [23]:
pokemon_list = pokedex_df['Pokémon'].tolist()
pokemon_name_corrections = {
    # Change names to match urls
    "nidoran♀": "nidoran-f",
    "nidoran♂": "nidoran-m",
    "mr. mime": "mr-mime",
    "mime jr.": "mime-jr",
    "farfetch'd": "farfetchd",
    # Replace normal forms with standard names used in urls
    "deoxys normal forme": "deoxys",
    "burmy plant cloak": "burmy",
    "wormadam plant cloak": "wormadam",
    "shaymin land forme": "shaymin",
    "basculin red-striped form": "basculin",
    "darmanitan standard mode": "darmanitan",
    "tornadus incarnate forme": "tornadus",
    "thundurus incarnate forme": "thundurus",
    "landorus incarnate forme": "landorus",
    "meloetta aria forme": "meloetta",
    "keldeo resolute form": "keldeo"
}
# Apply corrections to pokemon_list
pokemon_list = [pokemon_name_corrections.get(pokemon.lower(), pokemon) for pokemon in pokemon_list]
# Remove alternative forms from list, as they are not scrapable as their own urls
remove_list=['CASTFORM SUNNY FORM','CASTFORM RAINY FORM','CASTFORM SNOWY FORM','DEOXYS ATTACK FORME','DEOXYS DEFENSE FORME','DEOXYS SPEED FORME','BURMY SANDY CLOAK','BURMY TRASH CLOAK','WORMADAM SANDY CLOAK','WORMADAM TRASH CLOAK','ROTOM HEAT FORME','ROTOM WASH FORME','ROTOM FROST FORME','ROTOM FAN FORME','ROTOM MOW FORME','GIRATINA DISTORTION FORME','SHAYMIN SKY FORME','BASCULIN WHITE-STRIPED FORM','BASCULIN BLUE-STRIPED FORM','DARMANITAN ZEN MODE','TORNADUS THERIAN FORME','THUNDURUS THERIAN FORME','LANDORUS THERIAN FORME','MELOETTA PIROUETTE FORME','KYUREM WHITE','KYUREM BLACK','KELDEO RESOLUTE FORM']
for pokemon in remove_list:
    if pokemon in pokemon_list:
        pokemon_list.remove(pokemon)

#### FireRed/LeafGreen, Ruby/Sapphire/Emerald - Gen3

In [25]:
# This loop scrapes moveset data for hundreds of pokemon. Running this more than once for data analysis is computationally expensive and time-consuming. Therefore, this loop is only run once, then a dictionary containing each moveset dataframe is stored locally using pickle for further data anaylsis.
# Remove "if False" clause to run the loop for the first time
if False:
    gen3_movesets = {}
    for pokemon in pokemon_list:
        gen3_moveset_url=f'https://pokemondb.net/pokedex/{pokemon}/moves/3'
        df = scrape_table(gen3_moveset_url)
        # Break the loop once it reaches gen4 pokemon
        if df is None:
            break
        # Apply transformations
        df.columns = df.iloc[0]
        df = df.iloc[1:].rename(columns = {'Lv.':'Level Learned', 'Acc.':'Accuracy'})
        df = df.drop(['Cat.'], axis=1)
        df=df.loc[:df['Level Learned'].str.contains('Move|HM', na=False).idxmax() - 1]  # Cap table at level up movesets, ignore additional move data such as TMs/HMs
        gen3_movesets[pokemon] = df
    gen3_movesets = {key.lower(): value for key, value in gen3_movesets.items()}  # Convert dictionary keys to lowercase to make querying easier
    # Store gen3 pokemon movesets dictionary as a pkl file 
    with open('gen3_movesets.pkl', 'wb') as a:
        pickle.dump(gen3_movesets, a)

In [26]:
# Load the gen3 movesets dictionary from pkl file storage. 
# This will allow you to query all previously scraped tables for data analysis without having to run the loop again.
with open('gen3_movesets.pkl', 'rb') as a:
    gen3_movesets = pickle.load(a)

In [27]:
# Call gen3 pokemon moveset
gen3_movesets['deoxys']

,Level Learned,Move,Type,Power,Accuracy
1,1,Leer,Normal,—,100
2,1,Wrap,Normal,15,85
3,5,Night Shade,Ghost,—,100
4,10,Teleport,Psychic,—,—
5,15,Knock Off,Dark,20,100
6,20,Pursuit,Dark,40,100
7,25,Psychic,Psychic,90,100
8,30,Snatch,Dark,—,—
9,35,Cosmic Power,Psychic,—,—
10,40,Recover,Normal,—,—


#### Diamond/Pearl/Platinum, HeartGold/SoulSilver - Gen4

In [29]:
# This loop scrapes moveset data for hundreds of pokemon. Running this more than once for data analysis is computationally expensive and time-consuming. Therefore, this loop is only run once, then a dictionary containing each moveset dataframe is stored locally using pickle for further data anaylsis.
# Remove "if False" clause to run the loop for the first time
if False:
    gen4_movesets = {}
    for pokemon in pokemon_list:
        gen4_moveset_url=f'https://pokemondb.net/pokedex/{pokemon}/moves/4'
        df = scrape_table(gen4_moveset_url)
        # Break the loop once it reaches gen5 pokemon
        if df is None:
            break
        # Apply transformations
        df.columns = df.iloc[0]
        df = df.iloc[1:].rename(columns = {'Lv.':'Level Learned', 'Cat.':'Category', 'Att.':'Power', 'Acc':'Accuracy'})
        df['Category'] = df['Category'].replace('https://www.serebii.nethttps://img.pokemondb.net/images/icons/move-physical.png', 'Physical')\
                                .replace('https://www.serebii.nethttps://img.pokemondb.net/images/icons/move-special.png', 'Special')\
                                .replace('https://www.serebii.nethttps://img.pokemondb.net/images/icons/move-status.png', 'Other')
        df = df.loc[:df['Level Learned'].str.contains('Move|HM', na=False).idxmax() -1]  # Cap table at level up movesets, ignore additional move data such as TMs/HMs
        gen4_movesets[pokemon] = df
    gen4_movesets = {key.lower(): value for key, value in gen4_movesets.items()}  # Convert dictionary keys to lowercase to make querying easier
    # Store gen4 pokemon movesets dictionary as a pkl file 
    with open('gen4_movesets.pkl', 'wb') as b:
        pickle.dump(gen4_movesets, b)

In [30]:
# Load the gen4 movesets dictionary from pkl file storage. 
# This will allow you to query all previously scraped tables for data analysis without having to run the loop again.
with open('gen4_movesets.pkl', 'rb') as b:
    gen4_movesets = pickle.load(b)

In [31]:
# Call gen4 pokemon moveset
gen4_movesets['arceus']

,Level Learned,Move,Type,Category,Power,Acc.
1,1,Cosmic Power,Psychic,Other,—,—
2,1,Natural Gift,Normal,Physical,—,100
3,1,Punishment,Dark,Physical,—,100
4,1,Seismic Toss,Fighting,Physical,—,100
5,10,Gravity,Psychic,Other,—,—
6,20,Earth Power,Ground,Special,90,100
7,30,Hyper Voice,Normal,Special,90,100
8,40,ExtremeSpeed,Normal,Physical,80,100
9,50,Refresh,Normal,Other,—,—
10,60,Future Sight,Psychic,Special,80,90


#### Black/White, Black/White 2 - Gen5

In [33]:
# This loop scrapes moveset data for hundreds of pokemon. Running this more than once for data analysis is computationally expensive and time-consuming. Therefore, this loop is only run once, then a dictionary containing each moveset dataframe is stored locally using pickle for further data anaylsis.
# Remove "if False" clause to run the loop for the first time
if False:
    gen5_movesets = {}
    for pokemon in pokemon_list:
        gen5_moveset_url=f'https://pokemondb.net/pokedex/{pokemon}/moves/5' 
        df = scrape_table(gen5_moveset_url)
        # Apply transformations
        df.columns = df.iloc[0]
        df = df.iloc[1:].rename(columns = {'Lv.':'Level Learned', 'Cat.':'Category', 'Att.':'Power', 'Acc.':'Accuracy'})
        df['Category'] = df['Category'].replace('https://www.serebii.nethttps://img.pokemondb.net/images/icons/move-physical.png','Physical')\
                                .replace('https://www.serebii.nethttps://img.pokemondb.net/images/icons/move-special.png', 'Special')\
                                .replace('https://www.serebii.nethttps://img.pokemondb.net/images/icons/move-status.png', 'Other')
        df = df.loc[:df['Level Learned'].str.contains('Move|HM', na=False).idxmax() -1]
        gen5_movesets[pokemon] = df
    gen5_movesets = {key.lower(): value for key, value in gen5_movesets.items()}  # Convert dictionary keys to lowercase to make querying easier
    # Store gen5 pokemon movesets dictionary as a pkl file 
    with open('gen5_movesets.pkl', 'wb') as c:
        pickle.dump(gen5_movesets, c)

In [34]:
# Load the gen5 movesets dictionary from pkl file storage. 
# This will allow you to query all previously scraped tables for data analysis without having to run the loop again.
with open('gen5_movesets.pkl', 'rb') as c:
    gen5_movesets = pickle.load(c)

In [35]:
# Call gen5 pokemon moveset
gen5_movesets['genesect']

,Level Learned,Move,Type,Category,Power,Accuracy
1,1,Magnet Rise,Electric,Other,—,—
2,1,Metal Claw,Steel,Physical,50,95
3,1,Quick Attack,Normal,Physical,40,100
4,1,Screech,Normal,Other,—,85
5,1,Techno Blast,Normal,Special,85,100
6,7,Fury Cutter,Bug,Physical,20,95
7,11,Lock-On,Normal,Other,—,—
8,18,Flame Charge,Fire,Physical,50,100
9,22,Magnet Bomb,Steel,Physical,60,∞
10,29,Slash,Normal,Physical,70,100


##### Rather than looping through every pokemon and scraping hundreds of tables for each, you can define a function that scrapes moveset data only for pokemon of interest (saves memory/less cost intensive, better for interactive applications, however requires redundant scraping requests and is therefore less efficient if you want to perform data analysis)

In [37]:
if False:
    def gen3_moveset(pokemon):
        moveset_url=f'https://pokemondb.net/pokedex/{pokemon}/moves/3'
        scrape_table(moveset_url) 
        df.columns=df.iloc[0]
        df=df.iloc[1:].rename(columns={'Lv.':'Level','Power':'Att.'}).drop(['Cat.'], axis=1)
        df=df.loc[:df['Level'].str.contains('Move', na=False).idxmax() - 1]
        return df
    gen3_moveset('treecko')